In [1]:
import os
import re
import json
import gdal
import datetime
import itertools
import shapefile
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy import stats
from datetime import datetime
from itertools import product
import matplotlib.pyplot as plt
import functions_ground_motion as fgm

import sklearn.metrics as metrics
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

ModuleNotFoundError: No module named 'lsdfailtools'

In [2]:
with open("../file_with_paths.json") as file_with_paths :
    FILE_PATHS = json.load(file_with_paths)

print("The base output directory is {}".format(FILE_PATHS["ground_motion_csv"]))

ground_motion_dir = FILE_PATHS["ground_motion_csv"]
ground_motion_file = ground_motion_dir + ""
# for later: this is how the ground motion data have been saved
#out_dir_csv + 'Timeseries_GroundMotion_pixel'+str(i)+'_'+str(j)+'_failure.csv'


precip_dir = FILE_PATHS["rain_intensity_caliv_valid"]
out_dir = FILE_PATHS["time_series_ml"]

# Here comes the rain again
rain_dir = FILE_PATHS["rain_dir"]
rain_file = rain_dir + "2014-01-01_to_2019-12-31_Intensity.csv"

topo_dir = FILE_PATHS["topo_dir"]
slopefile = topo_dir + "eu_dem_AoI_epsg32633_SLOPE.bil"

# road file
roaddir = FILE_PATHS["road_dir"]
roadfile = roaddir + "Road_line.shp" # this is in EPSG:32633

merged_ground_motion_distance_dir = FILE_PATHS["ground_motion_csv"]
merged_ground_motion_distance_file = merged_ground_motion_distance_dir + "merged_result.csv"



The base output directory is /exports/csce/datastore/geos/groups/LSDTopoData/FORESEE/Data/ground_motion_csv/


In [3]:
concatenated_dir = FILE_PATHS["ground_motion_csv"]
concatenated_file = concatenated_dir + "combined_failure_pixels.csv"


In [4]:
# load the data
# Probably load the data on each of the csv files as we process it
# so that the data doesn't need to be stored in memory all the time.

# create the csv file with all the pixel locations
# fgm.make_pxl_csv(ground_motion_dir)

# concatenates all csv files
# fgm.concatenate_csv_files(ground_motion_dir)



# Read the road file
road = shapefile.Reader(roadfile)
roadline = np.array(road.shapes()[0].points)


slopearr, pixelWidth, (geotransform, inDs) = fgm.ENVI_raster_binary_to_2d_array(slopefile)




NameError: name 'fgm' is not defined

In [ ]:
########### read pixel position data ##################

ground_motion_pxl = pd.read_csv(ground_motion_dir + "pixel_values.csv",sep=',')
ground_motion_pxl = np.array(ground_motion_pxl)

# the first column only has indices - we don't need that.
ground_motion_pxl = ground_motion_pxl[:,1:]



In [ ]:
########### read concatenated ground motion data ##################
concat_ground_motion_pxl = pd.read_csv(concatenated_file)
concat_ground_motion_pxl = np.array(concat_ground_motion_pxl)




In [ ]:
########### read distance to road data ###############
road_distances = pd.read_csv(ground_motion_dir + "road_distances.csv",sep=',')
road_distances = np.array(road_distances)

# the first column only has indices - we don't need that.
road_distances = road_distances[:,1:]

# convert into dataframe so that we can merge later - not the keys have the be the same for the 2 df to merge.
road_distances_df = pd.DataFrame({'rows': road_distances[:,0],'cols': road_distances[:,1], 'distance_to_road':road_distances[:,2]})



In [ ]:
# convert timeseries into dataframe
concat_ground_motion_df = pd.DataFrame({'ground_motion': concat_ground_motion_pxl[:, 0], 'time_of_motion': concat_ground_motion_pxl[:, 1], 'slope': concat_ground_motion_pxl[:,2], 'rows': concat_ground_motion_pxl[:,3],'cols': concat_ground_motion_pxl[:,4]})
print(concat_ground_motion_df.head())

# merge the road distance and the ground motion timeseries dataframes.
result = pd.merge(concat_ground_motion_df, road_distances_df, how='inner', on=['rows', 'cols'])

result.to_csv(ground_motion_dir+'merged_result.csv')